#Libraries and Data

In [2]:
#import libraries
import numpy as np
import pandas as pd
import itertools
from prophet import Prophet
import yfinance
import pickle
import xgboost as xgb
from datetime import datetime

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [3]:
def fetch_stock_data(tickers):
    data = yfinance.download(tickers,period='1d')
    return data

def load_and_setup_data(sybmol):
    df = pd.read_csv("../stock_historical_data/{}.csv".format(sybmol))
    df.set_index("Date", inplace=True)
  

    #concatenated_df = pd.concat([df,new_data])
    return df

In [4]:

def fbprobetModel(stock_name,input_data):
    # Load the saved model from the file
    loaded_model = {}
    with open('../TrainedModel/prophet/{}_prophet_model_1.pkl'.format(stock_name), 'rb') as f:
        loaded_model = pickle.load(f)
    
    # Load the saved model from the file
    loaded_model_prophet = {}
    with open('../TrainedModel/prophet/{}prophet_model_2.pkl'.format(stock_name), 'rb') as f:
        loaded_model_prophet = pickle.load(f)
    
    data = {
        'ds' : [input_data["ds"]]
    }
  

    # Create a new DataFrame object
    data_fr = pd.DataFrame(data)
    data_fr.head()
    
    forecast = loaded_model_prophet.predict(data_fr)
    
    #get some variables
    df_xgb = forecast.loc[:, ["trend", "weekly",  "multiplicative_terms","yhat"]]
    # Create a dictionary with data
    data = {
        'High': [input_data["High"]],
        'Low': [input_data["Low"]],
        'y': [input_data["Close"]],
        'Volume': [input_data["Volume"]],
        'trend': df_xgb.head(1).trend.values,
        'weekly':df_xgb.head(1).weekly.values,
        'multiplicative_terms': df_xgb.head(1).multiplicative_terms.values,
    }
  #  print("point 1",data)


    # Create a new DataFrame object
    df = pd.DataFrame(data)
   # print("point 2",df.head())

    Future = xgb.DMatrix(df, label = df.y)
 #   print("point 3",df)

    f_predictions = pd.Series(loaded_model.predict(Future), name = "XGBoost")
    return f_predictions


In [5]:
tickers = ["OLECTRA.NS","LT.NS","CONCOR.NS","ELGIEQUIP.NS","IOC.NS","BEL.NS","TATAELXSI.NS","^NSEI"]
# Add the tickers you want to fetch data for
stock_data = fetch_stock_data(tickers)


[*********************100%***********************]  8 of 8 completed


In [6]:
stock_data

Adj Close                                                   \
                BEL.NS   CONCOR.NS ELGIEQUIP.NS     IOC.NS        LT.NS   
Date                                                                      
2023-07-12  127.650002  683.150024   583.200012  97.900002  2454.949951   

                                                         Close              \
             OLECTRA.NS TATAELXSI.NS         ^NSEI      BEL.NS   CONCOR.NS   
Date                                                                         
2023-07-12  1395.150024      7431.25  19384.300781  127.650002  683.150024   

            ...         Open                  Volume                         \
            ... TATAELXSI.NS         ^NSEI    BEL.NS CONCOR.NS ELGIEQUIP.NS   
Date        ...                                                               
2023-07-12  ...  7457.799805  19497.449219  18020824    828216      8436181   

                                                             
              IOC.NS    LT.NS OLECTRA.NS TATAELXSI.NS ^NSEI  
Date                                                         
2023-07-12  14969429  1822192    6720616        86133     0  

[1 rows x 48 columns]

In [7]:

result = []
Icdate = 0
Iresult = 1
IisClassification = 2
Imodel = 3
Istock = 4
Iclose = 5

current_date = datetime.now().strftime("%d-%m-%Y")

for ticker in tickers:
    stock_name = ticker
    if ticker == "^NSEI":
        stock_name = "NSEI"
        
    input_data = {
                    # "Open" : stock_data.head(1).Open[ticker].values[0],

                    "Close" : stock_data.head(1).Close[ticker].values[0],
                    "High" : stock_data.head(1).High[ticker].values[0],
                    "Low" : stock_data.head(1).Low[ticker].values[0], 
                    "Volume" : stock_data.head(1).Volume[ticker].values[0],
                    "ds":np.datetime_as_string(stock_data.index, unit='D')[0]
                }
    
   
    stock_name_only = stock_name.replace(".NS","")
    print(stock_name_only)
    print(input_data)

        
    model_result  = fbprobetModel(stock_name_only,input_data)
    print(model_result)
    result.append([current_date,model_result[0],0,'prophet',stock_name_only,stock_data.head(1).Close[ticker].values[0]])


OLECTRA
{'Close': 1395.1500244140625, 'High': 1419.0, 'Low': 1300.199951171875, 'Volume': 6720616, 'ds': '2023-07-12'}
0    967.595886
Name: XGBoost, dtype: float32
LT
{'Close': 2454.949951171875, 'High': 2485.949951171875, 'Low': 2447.800048828125, 'Volume': 1822192, 'ds': '2023-07-12'}
0    2470.361328
Name: XGBoost, dtype: float32
CONCOR
{'Close': 683.1500244140625, 'High': 684.0, 'Low': 672.0, 'Volume': 828216, 'ds': '2023-07-12'}
0    684.238892
Name: XGBoost, dtype: float32
ELGIEQUIP
{'Close': 583.2000122070312, 'High': 623.0, 'Low': 556.6500244140625, 'Volume': 8436181, 'ds': '2023-07-12'}
0    570.116211
Name: XGBoost, dtype: float32
IOC
{'Close': 97.9000015258789, 'High': 99.3499984741211, 'Low': 97.69999694824219, 'Volume': 14969429, 'ds': '2023-07-12'}
0    97.682907
Name: XGBoost, dtype: float32
BEL
{'Close': 127.6500015258789, 'High': 128.5, 'Low': 126.55000305175781, 'Volume': 18020824, 'ds': '2023-07-12'}
0    124.240364
Name: XGBoost, dtype: float32
TATAELXSI
{'Close': 

In [8]:
loaded_model = {}
with open('../TrainedModel/prophet/{}_prophet_model_1.pkl'.format("IOC"), 'rb') as f:
    loaded_model = pickle.load(f)

    

In [10]:
dir(loaded_model)

['__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_configure_constraints',
 '_get_feature_info',
 '_set_feature_info',
 '_transform_interaction_constraints',
 '_transform_monotone_constrains',
 '_validate_dmatrix_features',
 '_validate_features',
 'attr',
 'attributes',
 'best_iteration',
 'best_ntree_limit',
 'boost',
 'copy',
 'dump_model',
 'eval',
 'eval_set',
 'feature_names',
 'feature_types',
 'get_dump',
 'get_fscore',
 'get_score',
 'get_split_value_histogram',
 'handle',
 'inplace_predict',
 'load_config',
 'load_model',
 'num_boosted_rounds',
 'num_features',
 'predict

In [19]:
dataa = { "data":[]} 

for res in result:
    dataa["data"].append({ 
            "cdate": res[Icdate],
            "result": str(res[Iresult]),
            "pre_close" : str(res[Iclose]),            
            "isClassification": res[IisClassification],
            "model": res[Imodel],
            "prob": "0",
            "stock": res[Istock]})
    
dataa

{'data': [{'cdate': '02-07-2023',
   'result': '971.6514',
   'pre_close': '972.2000122070312',
   'isClassification': 0,
   'model': 'prophet',
   'stock': 'OLECTRA'},
  {'cdate': '02-07-2023',
   'result': '2470.3613',
   'pre_close': '2475.550048828125',
   'isClassification': 0,
   'model': 'prophet',
   'stock': 'LT'},
  {'cdate': '02-07-2023',
   'result': '662.8659',
   'pre_close': '661.7999877929688',
   'isClassification': 0,
   'model': 'prophet',
   'stock': 'CONCOR'},
  {'cdate': '02-07-2023',
   'result': '545.6747',
   'pre_close': '541.5999755859375',
   'isClassification': 0,
   'model': 'prophet',
   'stock': 'ELGIEQUIP'},
  {'cdate': '02-07-2023',
   'result': '91.81135',
   'pre_close': '91.30000305175781',
   'isClassification': 0,
   'model': 'prophet',
   'stock': 'IOC'},
  {'cdate': '02-07-2023',
   'result': '125.674614',
   'pre_close': '125.75',
   'isClassification': 0,
   'model': 'prophet',
   'stock': 'BEL'},
  {'cdate': '02-07-2023',
   'result': '7769.5

In [20]:
import requests
import json

url = "https://website-development-kerala.com/api_214124524/ai_model_daily_runner.php"

response = requests.post(url, json=dataa)
print(response.status_code)
print(response)

200
<Response [200]>
